In [158]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd


# Carregar os dados
df = pd.read_excel('brSerieA.xlsx')



# Criar o app
app = Dash(__name__)

# Anos disponíveis
anos_disponiveis = sorted(df['ano_campeonato'].unique())


# Função para calcular o time que mais pontuou
def time_mais_pontuou(df, ano):
    df = df[df['ano_campeonato'] == ano].copy()  # Evitar 'SettingWithCopyWarning'
    
    # Calcular vencedor de cada partida
    df['vencedor'] = df.apply(lambda row: row['time_mandante'] if row['gols_mandante'] > row['gols_visitante'] 
                            else row['time_visitante'] if row['gols_visitante'] > row['gols_mandante'] 
                            else 'Empate', axis=1)

    # Contabilizar vitórias
    df_vitorias = df[df['vencedor'] != 'Empate'].groupby('vencedor').size().reset_index(name='vitorias')

    # Calcular empates corretamente (para mandantes e visitantes)
    df_empates = df[df['gols_mandante'] == df['gols_visitante']]
    empates_por_time = df_empates.groupby('time_mandante').size().add(
        df_empates.groupby('time_visitante').size(), fill_value=0
    ).reset_index(name='empates')

    # Juntar os empates ao dataframe de vitórias
    df_vitorias = df_vitorias.merge(empates_por_time, left_on='vencedor', right_on='time_mandante', how='left').fillna(0)
    df_vitorias.drop(columns=['time_mandante'], inplace=True)
    df_vitorias['empates'] = df_vitorias['empates'].astype(int)  # Converter para inteiro

    # Adicionar coluna de pontos
    df_vitorias['pontos'] = df_vitorias['vitorias'] * 3 + df_vitorias['empates']

    # Remover colunas desnecessárias
    df_vitorias_sem_colunas = df_vitorias.drop(columns=['vitorias', 'empates'])

    # Ordenar do maior para o menor
    pontos_total = df_vitorias_sem_colunas.sort_values(by="pontos", ascending=False)

    return pontos_total


# Função para calcular o time que mais venceu
def time_mais_vitorias(df, ano):
    df_ano = df[df['ano_campeonato'] == ano]

    # Contar vitórias como mandante e visitante
    vitorias_mandante = df_ano[df_ano['gols_mandante'] > df_ano['gols_visitante']].groupby('time_mandante').size()
    vitorias_visitante = df_ano[df_ano['gols_visitante'] > df_ano['gols_mandante']].groupby('time_visitante').size()

    # Somar vitórias totais (fill_value=0 evita valores NaN)
    vitorias_total = vitorias_mandante.add(vitorias_visitante, fill_value=0)
    
    

    # Converter para DataFrame e ordenar
    df_vitorias = pd.DataFrame({'time': vitorias_total.index, 'vitorias': vitorias_total.values})
    df_vitorias = df_vitorias.sort_values(by="vitorias", ascending=False).reset_index(drop=True)
    display(df_vitorias)
    

    return df_vitorias    
    

# Função para verificar o melhor mandante
def melhor_mandante(df, ano):
    df_ano = df[df['ano_campeonato'] == ano]
    
    vitorias_mandante = df_ano[df_ano['gols_mandante'] > df_ano['gols_visitante']].groupby('time_mandante').size()
    pontos_total = vitorias_mandante.sort_values(ascending=False)
    
    df_melhor_casa = pd.DataFrame({'time': pontos_total.index, 'vitorias': pontos_total.values})
    df_melhor_casa = df_melhor_casa.sort_values(by="vitorias", ascending=False).reset_index(drop=True)
    
    display(df_melhor_casa)
    
    
    
    return df_melhor_casa

# Layout da aplicação
app.layout = html.Div(children=[
    html.H1(children='Campeonato Brasileiro Série A'),
    
    # Dropdown para selecionar o ano
    html.Label('Selecione o ano:'),
    dcc.Dropdown(
        id='ano-dropdown',
        options=[{'label': str(ano), 'value': ano} for ano in anos_disponiveis],   
        value=anos_disponiveis[0],  # Define o primeiro ano disponível como padrão
        clearable=False
    ),
    
    
    
    # Dropdown para selecionar a questão
    html.Label('Selecione a questão:'),
    dcc.Dropdown(
        id='questao-dropdown',
        options=[
            {'label': 'Time que mais pontuou', 'value': 'mais_pontuou'},
            {'label': 'Time que mais venceu', 'value': 'mais_venceu'},
            {'label': 'Melhor mandante', 'value': 'melhor mandante'},
        ],
        value='mais_pontuou',  # Define a primeira questão como padrão
        clearable=False
    ),
    
    # Gráfico de barras para mostrar o resultado
    dcc.Graph(id='grafico-resultados'),
])

# Callback para atualizar o gráfico baseado na seleção
@app.callback(
    Output('grafico-resultados', 'figure'),
    [Input('ano-dropdown', 'value'),
     Input('questao-dropdown', 'value')]
)
def exibir_grafico(ano, questao):
    if questao == 'mais_pontuou':
        pontos_total = time_mais_pontuou(df, ano)
        # Criação do gráfico de barras para os pontos totais
        
        fig = px.bar(
        pontos_total,
        x=pontos_total.vencedor,
        y=pontos_total.pontos,
        labels={'x': 'Time', 'y': 'Total de Pontos'},color="vencedor", color_discrete_sequence= px.colors.sequential.Plasma_r, 
        title=f'Pontos por time em {ano}'
        ).update_traces(textposition='inside', texttemplate='%{y}')

        fig.update_yaxes(showticklabels=False), 
        

        
    elif questao == 'mais_venceu':
        vitorias_total = time_mais_vitorias(df, ano)
        # Criação do gráfico de barras para as vitórias totais
        fig = px.bar(
            vitorias_total,
            x=vitorias_total.time,
            y=vitorias_total.vitorias,            
            labels={'x': 'Times', 'y': 'Total de Vitórias'}, color="time", color_discrete_sequence=px.colors.qualitative.Dark2,
            title=f'Vitórias por time em {ano}'
        )
        
    elif questao == 'melhor mandante':
        vitorias_mandante_total = melhor_mandante(df, ano)
        # Criação do gráfico de barras para as vitórias totais
        fig = px.bar(
            vitorias_mandante_total,
            x=vitorias_mandante_total.index,
            y=vitorias_mandante_total.values,            
            labels={'x': 'Times', 'y': 'Total de vitórias'},
            title=f'Vitórias por time em {ano}'
        )
    
    # Retornar a figura
    return fig

if __name__ == '__main__':
    app.run(debug=True)
